In [1]:
import datetime as dt
import pandas as pd
#!pip install yfinance
import yfinance as yf

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.sql.functions import to_date
from pyspark.ml.evaluation import RegressionEvaluator


In [3]:
# SparkSession başlatma
spark = SparkSession.builder.appName("NVDA Price Prediction").getOrCreate()

24/03/28 04:05:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
# Veriyi yükleme ve DataFrame oluşturma
data = spark.read.csv("file:///home/hduser/Desktop/NVDA/NVDA_histrical_data.csv", header=True, inferSchema=True)

# 'Date' sütununu tarih tipine dönüştürme 
data = data.withColumn("Date", to_date(data["Date"]))

In [5]:
data.show(5)

+----------+-------------------+-------------------+-------------------+-------------------+-------------------+--------+
|      Date|               Open|               High|                Low|              Close|          Adj Close|  Volume|
+----------+-------------------+-------------------+-------------------+-------------------+-------------------+--------+
|1999-03-22| 0.4466150104999542|0.44791701436042786|0.42447900772094727|0.42447900772094727| 0.3893754184246063| 3667200|
|1999-03-23|0.42708298563957214|0.42708298563957214|           0.390625| 0.3984380066394806|0.36548805236816406|16396800|
|1999-03-24|0.39583298563957214| 0.3984380066394806|0.38020798563957214|0.39583298563957214|0.36309847235679626| 6086400|
|1999-03-25| 0.3945310115814209|0.41666701436042786|0.39322900772094727|0.40104201436042786|0.36787667870521545| 4032000|
|1999-03-26|            0.40625|             0.4375|            0.40625|0.43619799613952637| 0.4001253545284271| 8827200|
+----------+------------

In [6]:
spark.createDataFrame(data.tail(5)).show()

+----------+------------------+------------------+------------------+------------------+------------------+--------+
|      Date|              Open|              High|               Low|             Close|         Adj Close|  Volume|
+----------+------------------+------------------+------------------+------------------+------------------+--------+
|2023-03-15|237.61000061035156|242.86000061035156|233.60000610351562|242.27999877929688|242.20230102539062|52448600|
|2023-03-16|240.27000427246094| 255.8800048828125|238.94000244140625|255.41000366210938|255.32810974121094|58325300|
|2023-03-17|259.82000732421875|  263.989990234375|256.67999267578125|            257.25|257.16754150390625|84854700|
|2023-03-20| 256.1499938964844|  260.239990234375| 251.3000030517578|             259.0| 258.9169616699219|43274700|
|2023-03-21|261.79998779296875| 263.9200134277344|253.80999755859375|  261.989990234375| 261.9059753417969|54740800|
+----------+------------------+------------------+--------------

In [7]:
# 'Close' dışındaki tüm sütunları at
df_org = data.select("Date", "Close")

# İlk 5 satırı göstermek için
df_org.show(5)


+----------+-------------------+
|      Date|              Close|
+----------+-------------------+
|1999-03-22|0.42447900772094727|
|1999-03-23| 0.3984380066394806|
|1999-03-24|0.39583298563957214|
|1999-03-25|0.40104201436042786|
|1999-03-26|0.43619799613952637|
+----------+-------------------+
only showing top 5 rows



In [8]:
df = df_org

In [9]:
from pyspark.sql.functions import col, count, when

# Her bir sütun için eksik değer sayısını hesaplama
missing_values = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
missing_values.show()

+----+-----+
|Date|Close|
+----+-----+
|   0|    0|
+----+-----+



In [10]:
# 'Close' sütunu ve 'Date' sütunu ile sınırlı DataFrame'i CSV olarak kaydet
#df.write.csv("file:///home/hduser/Desktop/NVDA/NVDA.csv", header=True)


Uyarıları tamamen kapatmanın bir yolu olmasa da, günlükleri daha az ayrıntılı hale getirerek uyarıların gösterilmesini azaltabilirsiniz. Bu, Spark'ın günlük seviyesini ayarlayarak yapılabilir. Örneğin, günlük seviyesini "ERROR" olarak ayarlamak yalnızca hata mesajlarının gösterilmesini sağlayacaktır.

In [11]:
spark.sparkContext.setLogLevel("ERROR")

1. Veri Setinin Yüklenmesi ve Öznitelik Oluşturma

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lag,col, to_date, dayofweek
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.window import Window
# Spark oturumunu başlatma
spark = SparkSession.builder.appName("Stock Price Forecasting").getOrCreate()

# Gecikmeli öznitelikler ekleyin
for i in range(1, 6):
    df = df.withColumn(f"lag_{i}", lag(col("Close"), i).over(Window.orderBy("Date")))

# Haftanın günü gibi tarih özniteliklerini ekleyin
df = df.withColumn("DayOfWeek", dayofweek(col("Date")))

# Eğitim için kullanılacak özniteliklerin listesini oluştur
input_cols = [f"lag_{i}" for i in range(1, 6)] + ['DayOfWeek']

In [13]:
from pyspark.sql import functions as F

for i in range(1, 6):
    # 'lag' sütunlarını mevcut 'Close' değeriyle doldurun
    df = df.withColumn(f"lag_{i}", F.coalesce(df[f"lag_{i}"], df["Close"]))

In [14]:
#df.show(6)

In [15]:
from pyspark.sql.functions import dayofweek
# 'DayOfWeek' sütunu için eksik değerleri doldurun
#df = df.withColumn('DayOfWeek', dayofweek(df['Date']))
# 'DayOfWeek' sütunu için `null` değerleri kontrol edin ve doldurun
#df = df.na.fill({'DayOfWeek': 0})

# VectorAssembler'ı tekrar çalıştırın
vectorAssembler = VectorAssembler(inputCols=input_cols, outputCol='features')
df = vectorAssembler.transform(df)

# Veri setini eğitim ve test olarak ayırma
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)


2. Model Eğitimi ve Katsayıların Gözlemlenmesi

In [16]:
from pyspark.ml.regression import LinearRegression

# Modeli tanımla ve eğit
lr = LinearRegression(featuresCol="features", labelCol="Close")
model = lr.fit(train_df)

# Eğitim ve test setleri üzerinde tahminler yap
train_predictions = model.transform(train_df)
test_predictions = model.transform(test_df)

# Model katsayılarını ve intercept değerini yazdır
print("Intercept:", model.intercept)
print("Coefficients:")
for i, coeff in enumerate(model.coefficients):
    print(f"lag_{i+1} coefficient: {coeff}")


Intercept: 0.09939249361496093
Coefficients:
lag_1 coefficient: 0.9663176389375417
lag_2 coefficient: 0.02625701686935515
lag_3 coefficient: 0.05823763429523521
lag_4 coefficient: -0.036756306638830706
lag_5 coefficient: -0.014273953896684722
lag_6 coefficient: -0.015020844606799031


3. Model Değerlendirme (RMSE ve MSE)

In [17]:
from pyspark.ml.evaluation import RegressionEvaluator

# RMSE ve MSE değerlendirici tanımla ve hesapla
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Close")

train_rmse = evaluator.evaluate(train_predictions, {evaluator.metricName: "rmse"})
test_rmse = evaluator.evaluate(test_predictions, {evaluator.metricName: "rmse"})
train_mse = evaluator.evaluate(train_predictions, {evaluator.metricName: "mse"})
test_mse = evaluator.evaluate(test_predictions, {evaluator.metricName: "mse"})

print(f"Train RMSE: {train_rmse}, Train MSE: {train_mse}")
print(f"Test RMSE: {test_rmse}, Test MSE: {test_mse}")


Train RMSE: 2.3269726507204274, Train MSE: 5.414801717200852
Test RMSE: 2.0887951595650978, Test MSE: 4.363065218622583


4. Gelecekteki 5 Gün İçin Tahminlerin Yapılması

In [18]:
from datetime import datetime, timedelta

# Burada son bilinen verileri kullanarak gelecekteki 5 gün için tahminler yapacağız.
# Kodun bu kısmı, son kapanış fiyatlarından yola çıkarak yeni öznitelikler oluşturur ve tahminleri yapar.

# Modelin en son tahminlerinden ve bilinen değerlerden yeni gecikmeli öznitelikler yaratma
last_row = train_df.orderBy(F.desc("Date")).first()
last_known_values = [last_row["Close"]] + [last_row[f"lag_{i}"] for i in range(1, 5)]

# Gelecekteki tarihler için tahminler yapma
future_dates = [last_row["Date"] + timedelta(days=i) for i in range(1, 6)]
future_predictions = []

In [19]:
#df.show(5)

In [20]:
future_dates

[datetime.date(2023, 3, 22),
 datetime.date(2023, 3, 23),
 datetime.date(2023, 3, 24),
 datetime.date(2023, 3, 25),
 datetime.date(2023, 3, 26)]

In [21]:
df

DataFrame[Date: date, Close: double, lag_1: double, lag_2: double, lag_3: double, lag_4: double, lag_5: double, DayOfWeek: int, features: vector]

In [24]:
from pyspark.sql.functions import lit, array
from pyspark.ml.linalg import Vectors

# Modeli ve son bilinen verileri yükleyin
#trained_model = # Eğitilmiş modelinizin yüklendiği yer
#last_known_values = # Son bilinen lag ve close değerleri

from datetime import datetime, timedelta
# Tahmin yapmak istediğiniz gün sayısını belirleyin
future_days = 5  # Örneğin, 5 gün sonrasını tahmin etmek istiyorsanız

from pyspark.sql.functions import to_date, max
# Veri setindeki en son tarihi al
last_date_row = df.select(max(to_date("Date"))).collect()
current_date = last_date_row[0][0]
#current_date = datetime.now()  # Mevcut tarihi kullanarak tahminler yapın

# Gelecek tarihler için boş bir DataFrame oluşturun
future_dates = [current_date + timedelta(days=i) for i in range(1, future_days + 1)]
future_predictions = []


# Örnek olarak, eğer modeliniz 5 gecikme özelliği kullanıyorsa (lag_1, lag_2, ... lag_5),
# ve last_known_values son bilinen 'Close' değeri ve 5 gecikme özelliğini içeriyorsa:
lag_features = ['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5']

# Bu değerleri kullanarak yeni özellikler vektörünü oluşturun
#new_features = [last_known_values[i] for i in lag_features]

# Döngü ile tahminleri yapın
for future_date in future_dates:
    # Yeni özellikler vektörünü oluşturun
    new_features = last_known_values[-len(lag_features):] + [last_known_values[0]]
    new_features_vector = Vectors.dense(new_features)
    
    # Yeni veri setini oluşturun ve tahmin yapın
    df_to_predict = spark.createDataFrame([(future_date, new_features_vector)], ["Date", "features"])
    prediction = model.transform(df_to_predict)
    
    # Tahmin edilen değeri alın ve sonuç listesine ekleyin
    predicted_close = prediction.select("prediction").collect()[0]["prediction"]
    future_predictions.append((future_date, predicted_close))
    
    # Sonraki tahmin için 'lag' değerlerini güncelleyin
    last_known_values = new_features[1:] + [predicted_close]

# Tahminleri içeren DataFrame'i oluşturun
predictions_df = spark.createDataFrame(future_predictions, ["Date", "predicted_close"])
predictions_df.show()


+----------+------------------+
|      Date|   predicted_close|
+----------+------------------+
|2023-03-22| 258.2656071960368|
|2023-03-23| 252.2940693102789|
|2023-03-24|239.18109114269188|
|2023-03-25|254.42228215756532|
|2023-03-26|247.37945246220337|
+----------+------------------+



In [25]:
spark.createDataFrame(df_org.tail(5)).show()

+----------+------------------+
|      Date|             Close|
+----------+------------------+
|2023-03-15|242.27999877929688|
|2023-03-16|255.41000366210938|
|2023-03-17|            257.25|
|2023-03-20|             259.0|
|2023-03-21|  261.989990234375|
+----------+------------------+

